In [ ]:
import zarrtraj
import MDAnalysis as mda
import zarr
from zarr.storage import LRUStoreCache
import s3fs
import os

# 1
yiipHDD = zarr.open_group("notebook_data_tmp/yiip.zarrtraj")

# 2
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

s3 = s3fs.S3FileSystem(key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)
store = s3fs.S3Map(root='zarrtraj-test-data/yiip.zarrtraj', s3=s3, check=False)
# Select max_size value in bytes based on chunking of zarrtraj data
# At least one chunk must fit in the cache
cache = LRUStoreCache(store, max_size=2**25)
yiipS3 = zarr.open_group(store=cache)

# 3
uHDD = mda.Universe("notebook_data_tmp/YiiP_system.pdb", yiipHDD)
uS3 = mda.Universe("notebook_data_tmp/YiiP_system.pdb", yiipS3)
uXTC = mda.Universe("notebook_data_tmp/YiiP_system.pdb", "YiiP_system_90ns_center.xtc")

In [ ]:
from MDAnalysis import transformations as trans



protein = u.select_atoms('protein')
water = u.select_atoms('resname SOL')

workflow = [trans.unwrap(u.atoms),  # unwrap all fragments
            trans.center_in_box(protein, # move atoms so protein
                                center='geometry'), # is centered
            trans.wrap(water, # wrap water back into box
                       compound='residues'), # keep each water whole
            trans.fit_rot_trans(protein, # align protein to first frame
                                protein,
                                weights='mass'),
           ]

u.trajectory.add_transformations(*workflow)

ow = u.select_atoms('name OW')
dens = density.DensityAnalysis(ow,
                               delta=4.0,
                               padding=2)
dens.run()